In [7]:

import pandas as pd

!pip install numerapi

from numerapi import NumerAPI

from pathlib import Path

#from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, f_regression


TOURNAMENT_NAME= "first_large"
ERA_COL = "era"
TARGET_COL = "target_nomi_20"
DATA_TYPE_COL = "data_type"
EXAMPLE_PREDS_COL = "example_preds"
PREDICTION_NAME = "prediction"


napi = NumerAPI()


In [8]:
from utils import save_stuff, load_stuff
# utilities


def score(df):
    return df[[TARGET_COL, PREDICTION_NAME]].corr(method="spearman")[TARGET_COL][PREDICTION_NAME]



In [11]:
current_round = napi.get_current_round(tournament=8)  # tournament 8 is the primary Numerai Tournament

print('Downloading dataset files...')

napi.download_dataset("numerai_tournament_data.parquet", f"tournament_data_{current_round}.parquet")
napi.download_dataset("numerai_validation_data.parquet", f"validation_data.parquet")
napi.download_dataset("example_validation_predictions.parquet", "example_validation_predictions.parquet")

model_name = f"model_target"
print(f"Checking for existing model '{model_name}'")
model = load_stuff(model_name)
selected_features = load_stuff('features')
if not model:
    print(f"model not found")

model_expected_features = selected_features

read_columns = model_expected_features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

validation_data = pd.read_parquet('validation_data.parquet', columns=read_columns)
validation_data[PREDICTION_NAME] = model.predict(validation_data[model_expected_features])

validation_preds = pd.read_parquet('example_validation_predictions.parquet')
validation_data[EXAMPLE_PREDS_COL] = validation_preds["prediction"]

validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and std {validation_correlations.std()}")


tournament_data = pd.read_parquet(f'tournament_data_{current_round}.parquet')

print('Predicting on tournament data')
tournament_data[PREDICTION_NAME] = model.predict(tournament_data[model_expected_features])

tournament_data[PREDICTION_NAME].to_csv("predictions.csv")





2021-12-05 03:27:50,243 INFO numerapi.utils: starting download
tournament_data_293.parquet: 582MB [00:21, 27.7MB/s]                            
2021-12-05 03:28:11,787 INFO numerapi.utils: starting download
validation_data.parquet: 228MB [00:08, 28.4MB/s]                            
2021-12-05 03:28:20,316 INFO numerapi.utils: target file already exists
2021-12-05 03:28:20,316 INFO numerapi.utils: download complete


Checking for existing model 'model_target'


ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [10]:
id = "OML65REYFDPC5O7N22XCRP44BG2M74XH"
key = "YSTL455VERL7WZ4D7OQ6XEYEQN2MRCCICBMILNFP3DUZC4MSAS2WSH2MV7ED6WB3"

napi = NumerAPI(public_id=id,secret_key=key)
#print('uploading')

path =  "predictions.csv"

print('uploading')
submission_string = napi.upload_predictions(file_path=path,version=2)
print(submission_string)






2021-12-05 03:27:31,180 INFO numerapi.base_api: uploading predictions...


uploading


2021-12-05 03:27:37,072 ERROR numerapi.base_api: Live prediction ids do not match. IDs must match current tournament data exactly, including ordering. Make sure you are using the latest tournament data.


ValueError: Live prediction ids do not match. IDs must match current tournament data exactly, including ordering. Make sure you are using the latest tournament data.